In [1]:
import sys
sys.path.append("../")

import os
from pathlib import Path
import wandb
import shortuuid

from confection import Config
from desci_sense.configs import init_config

In [ ]:
def log_pred_wandb(wandb_run, result):

    # get a unique ID for this prediction
    pred_uid = shortuuid.ShortUUID().random(length=8)

    artifact = wandb.Artifact(f"pred_{wandb_run.id}_{pred_uid}", type="prediction")

    columns = ["User", "URL", "Text", "Reasoning Steps", "Predicted Label", "True Label", "Tweet ID"]

    # extract relevant columns from prediction
    pred_row = [
        result['tweet']['user_name'],
        result['tweet']['tweetURL'],
        result['tweet']['text'],
        result['answer']['reasoning'],
        result['answer']['final_answer'],
        "", # no gold label meanwhile
        result['tweet']['tweetID']
    ]
    data = [pred_row]

    # add data to table
    table =  wandb.Table(data=data, columns=columns)
    artifact.add(table, "predictions")

    # log immediately since we don't know when user will close the session
    wandb.log_artifact(artifact)
    

    

    


In [2]:

shortuuid.ShortUUID().random(length=8)

'geKy3pHc'

In [3]:
config = init_config()

In [4]:
print(config.to_str())

[model]
model_name = "mistralai/mistral-7b-instruct"
temperature = 0.6

[prompt]
template_path = "desci_sense/prompting/templates/p1.txt"

[wandb]
wand_entity = "common-sense-makers"
wandb_db_name = "test-DB"


In [5]:
wandb_run = wandb.init(job_type="demo",project="st-demo", config=config, 
                                       entity=config["wandb"]["wand_entity"])

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ronentk (common-sense-makers). Use `wandb login --relogin` to force relogin


In [6]:
wandb_run.id

'skouh4x8'

In [7]:
artifact = wandb.Artifact(f"pred_table_{wandb_run.id}", type="prediction")

In [8]:
columns = ["User", "URL", "Text", "Reasoning Steps", "Predicted Label", "True Label", "Tweet ID" ,"UID"]

In [9]:
pred_1 = {'tweet': {'conversationID': '1719789465739333972',
  'date': 'Wed Nov 01 18:52:15 +0000 2023',
  'date_epoch': 1698864735,
  'hashtags': [],
  'likes': 91,
  'mediaURLs': ['https://pbs.twimg.com/media/F92_tj1asAAAoR-.jpg'],
  'media_extended': [{'altText': None,
    'size': {'height': 1279, 'width': 3199},
    'thumbnail_url': 'https://pbs.twimg.com/media/F92_tj1asAAAoR-.jpg',
    'type': 'image',
    'url': 'https://pbs.twimg.com/media/F92_tj1asAAAoR-.jpg'}],
  'possibly_sensitive': False,
  'qrtURL': None,
  'replies': 2,
  'retweets': 46,
  'text': 'My department at Univ Washington in Seattle is searching for a tenure-track assistant prof in "Quantitative Understanding of Collective Behavior" @UWBiology\n\nSee ad for more info about the search vision and to apply; happy to answer questions!\n\nhttps://apply.interfolio.com/130336 https://t.co/ydKZVuAeeY',
  'tweetID': '1719789465739333972',
  'tweetURL': 'https://twitter.com/bingbrunton/status/1719789465739333972',
  'user_name': 'Bing Wen Brunton',
  'user_screen_name': 'bingbrunton'},
 'answer': {'reasoning': '1. The post contains a job listing for a tenure-track assistant professor position.\n2. The position is in the field of "Quantitative Understanding of Collective Behavior" within the Department of Biology at the University of Washington in Seattle.\n3. The post includes a link to the job ad with more information and a call to apply.\n4. The post also includes a contact person for questions.\n\nCandidate Tags:\n\n* <job>: The post describes a job listing, specifically for a tenure-track assistant professor position in the field of "Quantitative Understanding of Collective Behavior" at the University of Washington.\n* <other>: This tag could be used if the post did not contain a clear job listing, but it is included here since the post clearly describes a job opportunity.',
  'final_answer': '<job>'}}

In [10]:
pred_1_row = [
    pred_1['tweet']['user_name'],
    pred_1['tweet']['tweetURL'],
    pred_1['tweet']['text'],
    pred_1['answer']['reasoning'],
    pred_1['answer']['final_answer'],
    "", # no gold label
    pred_1['tweet']['tweetID'],
    shortuuid.ShortUUID().random(length=8)
]
pred_1_row

['Bing Wen Brunton',
 'https://twitter.com/bingbrunton/status/1719789465739333972',
 'My department at Univ Washington in Seattle is searching for a tenure-track assistant prof in "Quantitative Understanding of Collective Behavior" @UWBiology\n\nSee ad for more info about the search vision and to apply; happy to answer questions!\n\nhttps://apply.interfolio.com/130336 https://t.co/ydKZVuAeeY',
 '1. The post contains a job listing for a tenure-track assistant professor position.\n2. The position is in the field of "Quantitative Understanding of Collective Behavior" within the Department of Biology at the University of Washington in Seattle.\n3. The post includes a link to the job ad with more information and a call to apply.\n4. The post also includes a contact person for questions.\n\nCandidate Tags:\n\n* <job>: The post describes a job listing, specifically for a tenure-track assistant professor position in the field of "Quantitative Understanding of Collective Behavior" at the Univer

In [11]:
data = [pred_1_row]

In [12]:
table =  wandb.Table(data=data, columns=columns)
artifact.add(table, "predictions")

In [13]:
artifact.add(table, "predictions")

ArtifactManifestEntry(path='predictions.table.json', digest='0+9p49HqNUoqg6gF7WjDSQ==', size=2331, local_path='/home/rkl25/.local/share/wandb/artifacts/staging/tmp9wfooy9i')

In [14]:
wandb.log_artifact(artifact)

<Artifact pred_table_skouh4x8>

In [15]:
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


In [12]:
ROOT = Path.cwd().parent 
cfg_path = ROOT / "etc" / "configs" / "sample_config.cfg"

In [13]:
cfg_path.exists()

True

In [3]:
config = Config().from_disk(str(cfg_path))

In [5]:
config["prompt"]["template_path"]

'desci_sense/prompting/templates/p1.txt'

In [10]:
prompt_path = ROOT / config["prompt"]["template_path"]
print(prompt_path.read_text())

You are an expert annotator who tags social media posts related to academic research, according to a predefined set of tags. 
The available tag types are:
<announce>: this post contains an announcement of new research. The announcement is likely made by the authors but may be a third party. The research should be a paper, dataset or other type of research output that is being announced publicly.
<review>: this post contains a review of another reference, such as a book, article or movie. The review could be positive or negative. A review can be detailed or a simple short endorsement.
<job>: this post describes a job listing, for example a call for graduate students or faculty applications.
<event>: this post describes an event, either real-world or an online event. Any kind of event is relevant, some examples of events could be seminars, meetups, or hackathons.
<other>: This is a special tag. Use this tag if none of the tags above are suitable. If you tag a post with <other>, no other 

In [2]:
wandb_run = wandb.init(job_type="demo",project="testing", 
                                       entity=WANDB_ENTITY)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ronentk (common-sense-makers). Use `wandb login --relogin` to force relogin


In [3]:
artifact = wandb.Artifact("my_table", type="dataset")

table = wandb.Table(
    columns=["a", "b", "c"], data=[(i, i * 2, 2**i) for i in range(10)]
)
artifact.add(table, "my_table")

wandb.log_artifact(artifact)

<Artifact my_table>

In [4]:
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


In [4]:
artifact = wandb.use_artifact("my_table:latest")

In [6]:
table = wandb.Table(
    columns=["d", "e", "f"], data=[(i, i * 2, 2**i) for i in range(10)]
)

In [7]:
artifact.add(table, "my_table")

ArtifactFinalizedError: 'Artifact' used on logged artifact. Can't modify finalized artifact.

In [8]:
tweet_example = {'conversationID': '1720443327663714559',
 'date': 'Fri Nov 03 14:10:28 +0000 2023',
 'date_epoch': 1699020628,
 'hashtags': [],
 'likes': 0,
 'mediaURLs': [],
 'media_extended': [],
 'possibly_sensitive': False,
 'qrtURL': 'https://twitter.com/i/status/1720425661813465096',
 'replies': 3,
 'retweets': 0,
 'text': 'I have no idea why this is not entirely obvious!  Are people just completely stupid!',
 'tweetID': '1720443327663714559',
 'tweetURL': 'https://twitter.com/IntuitMachine/status/1720443327663714559',
 'user_name': 'Carlos E. Perez',
 'user_screen_name': 'IntuitMachine'}



In [3]:
result = {'tweet': {'conversationID': '1719789465739333972',
  'date': 'Wed Nov 01 18:52:15 +0000 2023',
  'date_epoch': 1698864735,
  'hashtags': [],
  'likes': 91,
  'mediaURLs': ['https://pbs.twimg.com/media/F92_tj1asAAAoR-.jpg'],
  'media_extended': [{'altText': None,
    'size': {'height': 1279, 'width': 3199},
    'thumbnail_url': 'https://pbs.twimg.com/media/F92_tj1asAAAoR-.jpg',
    'type': 'image',
    'url': 'https://pbs.twimg.com/media/F92_tj1asAAAoR-.jpg'}],
  'possibly_sensitive': False,
  'qrtURL': None,
  'replies': 2,
  'retweets': 46,
  'text': 'My department at Univ Washington in Seattle is searching for a tenure-track assistant prof in "Quantitative Understanding of Collective Behavior" @UWBiology\n\nSee ad for more info about the search vision and to apply; happy to answer questions!\n\nhttps://apply.interfolio.com/130336 https://t.co/ydKZVuAeeY',
  'tweetID': '1719789465739333972',
  'tweetURL': 'https://twitter.com/bingbrunton/status/1719789465739333972',
  'user_name': 'Bing Wen Brunton',
  'user_screen_name': 'bingbrunton'},
 'answer': {'reasoning': '1. The post contains a job listing for a tenure-track assistant professor position.\n2. The position is in the field of "Quantitative Understanding of Collective Behavior" within the Department of Biology at the University of Washington in Seattle.\n3. The post includes a link to the job ad with more information and a call to apply.\n4. The post also includes a contact person for questions.\n\nCandidate Tags:\n\n* <job>: The post describes a job listing, specifically for a tenure-track assistant professor position in the field of "Quantitative Understanding of Collective Behavior" at the University of Washington.\n* <other>: This tag could be used if the post did not contain a clear job listing, but it is included here since the post clearly describes a job opportunity.',
  'final_answer': '<job>'}}

In [10]:
answer = {'reasoning': '1. The post contains a job listing for a tenure-track assistant professor position.\n2. The position is in the field of "Quantitative Understanding of Collective Behavior" within the Department of Biology at the University of Washington in Seattle.\n3. The post includes a link to the job ad with more information and a call to apply.\n4. The post also includes a contact person for questions.\n\nCandidate Tags:\n\n* <job>: The post describes a job listing, specifically for a tenure-track assistant professor position in the field of "Quantitative Understanding of Collective Behavior" at the University of Washington.\n* <other>: This tag could be used if the post did not contain a clear job listing, but it is included here since the post clearly describes a job opportunity.',
  'final_answer': '<job>'}

In [9]:
wandb_run.log(tweet_example)

In [4]:
wandb_run.log(result)

In [5]:
wandb_run.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job
